In [ ]:
import pandas as pd
import seaborn as sns
import statistics as stat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

df_original = pd.read_csv("https://raw.githubusercontent.com/alvarofavale/week7_ml/refs/heads/main/data/raw/train.csv")


In [ ]:
df = df_original.copy()
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df.describe()

In [ ]:
dfc = df[df["customer_id"] == 3392]
dfc

In [ ]:
df.credit_score.unique()

In [ ]:
df.columns

In [ ]:
df.drop(columns=["ssn", "name"], inplace = True, axis = 1)

In [ ]:
df.drop_duplicates()

In [ ]:
df_limit = df.sample(1000)
#sns.pairplot(df_limit, hue ="Credit_Score")

In [ ]:
features = df_limit.drop(columns = ["credit_score"])
features = features.select_dtypes(include = "number")
target = df_limit["credit_score"]

In [ ]:
target.replace({"Good" : 1, "Standard" : 2, "Poor" : 3})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [ ]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

In [ ]:
scaler = StandardScaler()

scaler.fit(X_train)

In [ ]:
X_train_norm_np = normalizer.transform(X_train)

X_test_norm_np = normalizer.transform(X_test)

In [ ]:
X_train_norm_df = pd.DataFrame(X_train_norm_np, columns = X_train.columns, index=X_train.index)
X_train_norm_df.head()

In [ ]:
X_test_norm_df = pd.DataFrame(X_test_norm_np, columns = X_test.columns, index=X_test.index)
X_test_norm_df.head()

In [ ]:
X_train_standarized_np = scaler.transform(X_train)
X_test_standarized_np = scaler.transform(X_test)

X_train_standarized_df = pd.DataFrame(X_train_standarized_np, columns = X_train.columns, index=X_train.index)
X_test_standarized_df  = pd.DataFrame(X_test_standarized_np, columns = X_test.columns, index=X_test.index)


DECISION TREE

In [ ]:
tree = DecisionTreeRegressor(max_depth=10)

- Training the model

Note: Decision Trees are not "distance" based models. Therefore, we don't need to have the data before training them. If we do so, the model with adapt to the new transformed data and the changes in the predictions will be small.

Here, for simplicity we will train a Decision Tree with transformed data

In [ ]:
y_train.replace({"Good" : 1, "Standard" : 2, "Poor" : 3}, inplace=True)

In [ ]:
y_train

In [ ]:
tree.fit(X_train_norm_df, y_train)

- Evaluate the model

In [ ]:
X_train_norm_df

In [ ]:
y_test.replace({"Good" : 1, "Standard" : 2, "Poor" : 3}, inplace=True)
y_test

In [ ]:
y_pred_test_dt = tree.predict(X_test_norm_df)

print(f"MAE, {mean_absolute_error(y_pred_test_dt, y_test): .2f}")
print(f"RMSE, {mean_squared_error(y_pred_test_dt, y_test, squared=False): .2f}")
print(f"R2 score, {tree.score(X_test_norm_df, y_test): .2f}")

Often we check what are the most relevant features, like we did before in Linear Regression. However, here we don't have coefficients. Therefore, to do this kind of analisys, we check in which order were the features selected to split the data. The intuition behind this method is that the most important features will be selected first, while less important fetures will be used in later splits.

Fortunatelly for us, this information in stored in the instance of the class and it can be accessed trough the attribute `.feature_importances_` once the model has been trained.

In [ ]:
tree_importance = {feature : importance for feature, importance in zip(X_train_norm_df.columns, tree.feature_importances_)}
tree_importance

In [ ]:
from sklearn.tree import export_text

tree_viz = export_text(tree, feature_names=list(X_train_norm_df.columns))
print(tree_viz)


A bit overwhelming to see, let's use graphviz library.

**Note**: you will need to install graphivz

* pip install graphviz
* conda install graphviz -y

depending on your environment and package mannager.

In [ ]:
#!pip install graphviz

- We will train a decision tree, in this case with max_depth=2 to better see the diagram

In [ ]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz

tree = DecisionTreeRegressor(max_depth=2)
tree.fit(X_train_norm_df, y_train)


dot_data = export_graphviz(tree, out_file="tree.dot", filled=True, rounded=True, feature_names=X_train_norm_df.columns)

with open("tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)